# Final Project Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import os, sys
import pandas as pd
import io
import requests
import warnings
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
! pip install cloudmesh-common -U

from cloudmesh.common.Benchmark import Benchmark

     |████████████████████████████████| 81kB 2.0MB/s 
     |████████████████████████████████| 133kB 7.9MB/s 
  Created wheel for python-hostlist: filename=python_hostlist-1.21-cp36-none-any.whl size=38932 sha256=0247fed1c86d73f7925d91d1fc8d3939791e6e3d14af9900606abee706210a78
  Stored in directory: /root/.cache/pip/wheels/0b/5b/55/ddcf52288f0b10f4564ca1b2531594ff7ccc65f487ba8dc437
Successfully built python-hostlist


In [3]:
! pip install utils
import utils

Now that the funtions have been imported the team can focus on the download coding. The following cells will set up an install for Kaggle files and prompt for an upload of the kaggle.json file for credentials. 

The mkdir function creates a directory for the Kaggle data. This cell will allow the team to verify that the kaggle.json file appropriately uploaded to the directory.

In [4]:
##import the kaggle.json from local to drive
!pip install -q kaggle
from google.colab import files
##when it asks you to choose a file select the kaggle.json located within the 'project' folder from the github repo
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hemmerleingavin","key":"a0412c93ecb33476babdf128d77136c9"}'}

In [6]:
##make a kaggle and a data folder
#!mkdir ~/.kaggle
!mkdir data
##copy the kaggle.json to the .kaggle folder then grant permissions
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#test to see if kaggle is working, should print list of datasets
!kaggle datasets list

mkdir: cannot create directory ‘data’: File exists
ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
nehaprabhavalkar/indian-food-101                                  Indian Food 101                                        7KB  2020-09-30 06:23:43           4817  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08           1184  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           5217  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-

Now, the team must download all of the datasets for the class. The three datasets are focused on the NBA. 

The first dataset is for injuries. Each injury will be used to set up players, timeframes, and severity of injuries. 

The other two datasets are for the player performance. By cross referencing this data to the previous list, the team will be able to see which players are limited from the injury and how performance is hampered by time in rehab.

In [7]:
##downloading all the datasets
!kaggle datasets download -d ghopkins/nba-injuries-2010-2018
!kaggle datasets download -d nathanlauga/nba-games
!kaggle datasets download -d pablote/nba-enhanced-stats

  0% 0.00/226k [00:00<?, ?B/s]
100% 226k/226k [00:00<00:00, 83.9MB/s]
 50% 9.00M/18.1M [00:01<00:02, 4.70MB/s]
100% 18.1M/18.1M [00:01<00:00, 10.2MB/s]
 54% 9.00M/16.7M [00:01<00:01, 7.51MB/s]
100% 16.7M/16.7M [00:01<00:00, 11.1MB/s]


In [8]:
##unzipping to the data folder
!unzip nba-injuries-2010-2018.zip -d data
!unzip nba-games.zip -d data
!unzip nba-enhanced-stats.zip -d data

Archive:  nba-injuries-2010-2018.zip
  inflating: data/injuries_2010-2020.csv  
Archive:  nba-games.zip
  inflating: data/games.csv          
  inflating: data/games_details.csv  
  inflating: data/players.csv        
  inflating: data/ranking.csv        
  inflating: data/teams.csv          
Archive:  nba-enhanced-stats.zip
  inflating: data/2012-18_officialBoxScore.csv  
  inflating: data/2012-18_playerBoxScore.csv  
  inflating: data/2012-18_standings.csv  
  inflating: data/2012-18_teamBoxScore.csv  
  inflating: data/2016-17_officialBoxScore.csv  
  inflating: data/2016-17_playerBoxScore.csv  
  inflating: data/2016-17_standings.csv  
  inflating: data/2016-17_teamBoxScore.csv  
  inflating: data/2017-18_officialBoxScore.csv  
  inflating: data/2017-18_playerBoxScore.csv  
  inflating: data/2017-18_standings.csv  
  inflating: data/2017-18_teamBoxScore.csv  
  inflating: data/metadata_officialBoxScore.pdf  
  inflating: data/metadata_playerBoxScore.pdf  
  inflating: data/metadata

The team must now use these downloads to create dataframes. Pandas dataframes will be easier to manage the data. The team will be able to use Pandas to process the data and allow the team to make correlations for feature engineering to create the models.

In [9]:
#create a list for each data set
ds_NBA_Injuries, ds_NBA_Games, ds_NBA_Enhanced = [], [], []

#import csv files as dataframes and save to respective list, injury set first
df_Injuries = pd.read_csv('data/injuries_2010-2020.csv')
df_Injury_Start = df_Injuries[df_Injuries.Acquired.isnull()]
df_Injury_End = df_Injuries[df_Injuries.Relinquished.isnull()]
ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('data/games.csv')
df_Games_gamesDetails = pd.read_csv('data/games_details.csv')
df_Games_players = pd.read_csv('data/players.csv')
df_Games_ranking = pd.read_csv('data/ranking.csv')
df_Games_teams = pd.read_csv('data/teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]
#nba enhanced stats dataset
df_En_officialBS_1218 = pd.read_csv('data/2012-18_officialBoxScore.csv')
df_En_playerBS_1218 = pd.read_csv('data/2012-18_playerBoxScore.csv')
df_En_standings_1218 = pd.read_csv('data/2012-18_standings.csv')
df_En_teamBS_1218 = pd.read_csv('data/2012-18_teamBoxScore.csv')  
df_En_officialBS_1617 = pd.read_csv('data/2016-17_officialBoxScore.csv')  
df_En_playerBS_1617 = pd.read_csv('data/2016-17_playerBoxScore.csv')
df_En_standings_1617 = pd.read_csv('data/2016-17_standings.csv')
df_En_teamBS_1617 = pd.read_csv('data/2016-17_teamBoxScore.csv')  
df_En_officialBS_1718 = pd.read_csv('data/2017-18_officialBoxScore.csv')  
df_En_playerBS_1718 = pd.read_csv('data/2017-18_playerBoxScore.csv')
df_En_standings_1718 = pd.read_csv('data/2017-18_standings.csv')
df_En_teamBS_1718 = pd.read_csv('data/2017-18_teamBoxScore.csv')  
##data/metadata_officialBoxScore.pdf, data/metadata_playerBoxScore.pdf, data/metadata_standing.pdf, data/metadata_teamBoxScore.pdf  
df_En_teamBS = pd.read_csv('data/teamBoxScore.csv')
ds_NBA_Enhanced = [df_En_officialBS_1218, df_En_officialBS_1617, df_En_officialBS_1718, df_En_playerBS_1218, df_En_playerBS_1617, df_En_playerBS_1718, df_En_standings_1218, df_En_standings_1617, df_En_standings_1718, \
                       df_En_teamBS_1218, df_En_teamBS_1617, df_En_teamBS_1718, df_En_teamBS]


#probably need some more data exploration and some feature engineering

## Exploratory Data Analysis

At this point, it is time to build into new useful sets of data. The team will explore different sets to combine in to the models to be trained.

In [10]:
#distinct player and player ID list
df_distinct_playerID = df_Games_players[["PLAYER_NAME", "PLAYER_ID"]].drop_duplicates()
df_distinct_playerID.astype({'PLAYER_ID':'object'}).dtypes
#distinct gameID and game date list
df_Games_games['GAME_DATE_EST'] = pd.to_datetime(df_Games_games['GAME_DATE_EST'])
df_distinct_gameId_date = df_Games_games[["GAME_ID", "GAME_DATE_EST"]].drop_duplicates()
#join player ID, for j=injury start db
df_Injury_Start = df_Injury_Start.join(df_distinct_playerID.astype('object').set_index('PLAYER_NAME'), on='Relinquished')
df_Injury_Start = df_Injury_Start.merge(df_Games_teams[["TEAM_ID", "NICKNAME"]], left_on="Team", right_on="NICKNAME")
df_Injury_Start.drop(['NICKNAME'], axis=1)
df_Injury_Start['Date']= pd.to_datetime(df_Injury_Start['Date'])
#again for injury end db
df_Injury_End = df_Injury_End.join(df_distinct_playerID.astype('object').set_index('PLAYER_NAME'), on='Acquired')
df_Injury_End = df_Injury_End.merge(df_Games_teams[["TEAM_ID", "NICKNAME"]], left_on="Team", right_on="NICKNAME")
df_Injury_End.drop(['NICKNAME'], axis=1)
df_Injury_End['Date']= pd.to_datetime(df_Injury_Start['Date'])
# df_distinct_playerID=df_distinct_playerID.sort_values('PLAYER_NAME')
df_Games_gamesDetails = df_Games_gamesDetails.merge(df_distinct_gameId_date, on="GAME_ID")

# df_distinct_playerID = df_distinct_playerID.sort_values(by=['PLAYER_NAME']).reset_index(drop=True, inplace=True)
#df_Injury_End

In [11]:
###TRYING TO SOLVE MINUTES FORMATTING


#df_Games_gamesDetails[['MIN']].find(':')
#int(inj_game['MIN'][:(inj_game['MIN'].find(':'))]) + (int(inj_game['MIN'][(inj_game['MIN'].find(':') +1):])/ 60)
df_Games_gamesDetails['MIN'] = int(df_Games_gamesDetails['MIN'][:(df_Games_gamesDetails['MIN'].find(':'))]) + (int(df_Games_gamesDetails['MIN'][(df_Games_gamesDetails['MIN'].find(':') +1):])/ 60)

AttributeError: ignored

In [12]:
#x = game_set1['GAME_ID']
#x
#y = df_Games_gamesDetails.loc[df_Games_gamesDetails['GAME_ID'].isin(x.values.tolist())]
game_set2
p_id
game_set2.loc[game_set2['PLAYER_ID'].astype('int') == 2430]
#game_set2['PLAYER_ID'].dtype
#y.iloc[0]
#temp.head(10)
x = df_Games_gamesDetails.loc[(df_Games_gamesDetails['PLAYER_ID'] == row.PLAYER_ID) & (df_Games_gamesDetails['GAME_DATE_EST'] < row.Date)].nlargest(5, 'GAME_DATE_EST')[['MIN']]
df_Injury_Start.loc[df_Injury_Start['Relinquished'] == "Omer Asik"]
prior5['PLUS_MINUS'].mean()

NameError: ignored

###Feature Engineering for Injury sets
#####Goal is to have stats for injury game, average stats of last/first 5 games and maybe join season avg?

In [13]:
#df_Injury_Start.Date

for index, row in df_Injury_Start.iterrows():
        #games of just that player
        temp = df_Games_gamesDetails.loc[df_Games_gamesDetails['PLAYER_ID'] == row.PLAYER_ID]
        #games before and inlucding injury date
        inj_game = temp.loc[(temp['GAME_DATE_EST'] == row.Date)]
        #5 games prior and the game of injury, for some reason we need to have 4 different variabels, did not work with resetting the variable 'game_set' to itself
        temp2 = temp.loc[(temp['GAME_DATE_EST'] <= row.Date)]
        game_set = temp2.nlargest(6, 'GAME_DATE_EST')
        if len(game_set) > 0:
          #injury game
          inj_game = game_set.iloc[0]
          #5 games prior to injury
          prior5 = game_set.iloc[1:]
          #storing game data from injury game
          df_Injury_Start['inj_MIN'] = inj_game[['MIN']]
          df_Injury_Start['inj_FGA'] = inj_game[['FGA']]
          df_Injury_Start['inj_FG_PCT'] = inj_game[['FG_PCT']]
          df_Injury_Start['inj_FG3A'] = inj_game[['FG3A']]
          df_Injury_Start['inj_FG3_PCT'] = inj_game[['FG3_PCT']]
          df_Injury_Start['inj_FTA'] = inj_game[['FTA']]
          df_Injury_Start['inj_FT_PCT'] = inj_game[['FT_PCT']]
          df_Injury_Start['inj_REB'] = inj_game[['REB']]
          df_Injury_Start['inj_AST'] = inj_game[['AST']]
          df_Injury_Start['inj_STL'] = inj_game[['STL']]
          df_Injury_Start['inj_BLK'] = inj_game[['BLK']]
          df_Injury_Start['inj_TO'] = inj_game[['TO']]
          df_Injury_Start['inj_PF'] = inj_game[['PF']]
          df_Injury_Start['inj_PTS'] = inj_game[['PTS']]
          df_Injury_Start['inj_PLUS_MINUS'] = inj_game[['PLUS_MINUS']]
          #storing game data from prior 5 games
          df_Injury_Start['p5_MIN'] = prior5[['MIN']].mean()
          df_Injury_Start['p5_FGA'] = prior5[['FGA']].mean()
          df_Injury_Start['p5_FG_PCT'] = prior5[['FG_PCT']].mean()
          df_Injury_Start['p5_FG3A'] = prior5[['FG3A']].mean()
          df_Injury_Start['p5_FG3_PCT'] = prior5[['FG3_PCT']].mean()
          df_Injury_Start['p5_FTA'] = prior5[['FTA']].mean()
          df_Injury_Start['p5_FT_PCT'] = prior5[['FT_PCT']].mean()
          df_Injury_Start['p5_REB'] = prior5[['REB']].mean()
          df_Injury_Start['p5_AST'] = prior5[['AST']].mean()
          df_Injury_Start['p5_STL'] = prior5[['STL']].mean()
          df_Injury_Start['p5_BLK'] = prior5[['BLK']].mean()
          df_Injury_Start['p5_TO'] = prior5[['TO']].mean()
          df_Injury_Start['p5_PF'] = prior5[['PF']].mean()
          df_Injury_Start['p5_PTS'] = prior5[['PTS']].mean()
          df_Injury_Start['p5_PLUS_MINUS'] = prior5[['PLUS_MINUS']].mean()


        #print(inj_game)
        #print(prior5)

KeyboardInterrupt: ignored

In [14]:
#df_Injury_End

for index, row in df_Injury_End.iterrows():
        #games of just that player
        temp = df_Games_gamesDetails.loc[df_Games_gamesDetails['PLAYER_ID'] == row.PLAYER_ID]
        #games before and inlucding injury date
        temp2 = temp.loc[(temp['GAME_DATE_EST'] >= row.Date)]
        #5 games prior and the game of injury, for some reason we need to have 4 different variabels, did not work with resetting the variable 'game_set' to itself
        game_set = temp.nsmallest(6, 'GAME_DATE_EST')
        if len(game_set) > 0:
          #injury game
          inj_game = game_set.iloc[0]
          #5 games post injury
          post5 = game_set.iloc[1:]
          #storing game data from injury game
          df_Injury_End['inj_MIN'] = inj_game[['MIN']]
          df_Injury_End['inj_FGA'] = inj_game[['FGA']]
          df_Injury_End['inj_FG_PCT'] = inj_game[['FG_PCT']]
          df_Injury_End['inj_FG3A'] = inj_game[['FG3A']]
          df_Injury_End['inj_FG3_PCT'] = inj_game[['FG3_PCT']]
          df_Injury_End['inj_FTA'] = inj_game[['FTA']]
          df_Injury_End['inj_FT_PCT'] = inj_game[['FT_PCT']]
          df_Injury_End['inj_REB'] = inj_game[['REB']]
          df_Injury_End['inj_AST'] = inj_game[['AST']]
          df_Injury_End['inj_STL'] = inj_game[['STL']]
          df_Injury_End['inj_BLK'] = inj_game[['BLK']]
          df_Injury_End['inj_TO'] = inj_game[['TO']]
          df_Injury_End['inj_PF'] = inj_game[['PF']]
          df_Injury_End['inj_PTS'] = inj_game[['PTS']]
          df_Injury_End['inj_PLUS_MINUS'] = inj_game[['PLUS_MINUS']]
          #storing game data from prior 5 games
          df_Injury_End['p5_MIN'] = post5[['MIN']].mean()
          df_Injury_End['p5_FGA'] = post5[['FGA']].mean()
          df_Injury_End['p5_FG_PCT'] = post5[['FG_PCT']].mean()
          df_Injury_End['p5_FG3A'] = post5[['FG3A']].mean()
          df_Injury_End['p5_FG3_PCT'] = post5[['FG3_PCT']].mean()
          df_Injury_End['p5_FTA'] = post5[['FTA']].mean()
          df_Injury_End['p5_FT_PCT'] = post5[['FT_PCT']].mean()
          df_Injury_End['p5_REB'] = post5[['REB']].mean()
          df_Injury_End['p5_AST'] = post5[['AST']].mean()
          df_Injury_End['p5_STL'] = post5[['STL']].mean()
          df_Injury_End['p5_BLK'] = post5[['BLK']].mean()
          df_Injury_End['p5_TO'] = post5[['TO']].mean()
          df_Injury_End['p5_PF'] = post5[['PF']].mean()
          df_Injury_End['p5_PTS'] = post5[['PTS']].mean()
          df_Injury_End['p5_PLUS_MINUS'] = post5[['PLUS_MINUS']].mean()


        #print(inj_game)
        #print(prior5)

KeyboardInterrupt: ignored

# Part 2 Building the Keras Model

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/). We are first going to set up our Benchmark Test to be used when we are Benchmarking our model.

In [15]:
def b():
  Benchmark.Start()
  print ("b")
  import time
  time.sleep(3)
  Benchmark.Stop()

def c():
  Benchmark.Start()
  print ("c")
  import time
  time.sleep(1)
  Benchmark.Stop()

In [16]:
 b()
 c()

 Benchmark.print()

b
c

+---------------------+------------------------------------------------------------------+
| Attribute           | Value                                                            |
|---------------------+------------------------------------------------------------------|
| BUG_REPORT_URL      | "https://bugs.launchpad.net/ubuntu/"                             |
| DISTRIB_CODENAME    | bionic                                                           |
| DISTRIB_DESCRIPTION | "Ubuntu 18.04.5 LTS"                                             |
| DISTRIB_ID          | Ubuntu                                                           |
| DISTRIB_RELEASE     | 18.04                                                            |
| HOME_URL            | "https://www.ubuntu.com/"                                        |
| ID                  | ubuntu                                                           |
| ID_LIKE             | debian                                                       

Now that we know which GPU we are using, we can get into the actual work. The following is building our Keras model.

In [17]:
np.random.seed(23)
warnings.filterwarnings("ignore")

In [18]:
df_baseline = df_Injury_Start

#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=['Relinquished']).reset_index(drop=True, inplace=True)
# df_baseline['FPTS_pred'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

In [19]:
df_baseline

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,TEAM_ID,NICKNAME,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2430,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-10-27,Bulls,NaN,Carlos Boozer,placed on IL with fractured bone in right pink...,2430,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-11-24,Bulls,NaN,Taj Gibson,right ankle/foot injury (DNP),201959,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-11-26,Bulls,NaN,Derrick Rose,placed on IL with sore neck,201565,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-11-27,Bulls,NaN,John Lucas III,placed on IL,101249,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16889,2020-03-08,Pelicans,NaN,Nickeil Alexander-Walker,placed on IL with fractured right wrist,1629638,1610612740,Pelicans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16890,2020-08-07,Pelicans,NaN,Zion Williamson,placed on IL,1629627,1610612740,Pelicans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16891,2020-08-10,Pelicans,NaN,Brandon Ingram,sore right knee (out for season),1627742,1610612740,Pelicans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16892,2020-08-10,Pelicans,NaN,Jrue Holiday,placed on IL with right elbow injury (out for ...,201950,1610612740,Pelicans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=['Relinquished']).reset_index(drop=True, inplace=True)


# df_baseline['	PLAYER_ID'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

# Part No X. Building the Model

The team is now moving on to building the model for the baseline. Linear Regression can be used to model the values. Additionally, a Random Forest modeling function was used to verify model performance. 

In [54]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

The pipeline will be built off of using df_baseline.PLAYER_ID. To change the data going into the model, the team had to modify the dataframe input to get to the results. Comments were put around to make it easy to find the code in the Notebook.

In [50]:
#df_baseline = df_baseline
# basic =  ['PTS','3P','AST','TRB','STL','BLK','TOV', 'DD', 'TD']

###################################
##                               ##
##     INSERT CODE               ##
##                               ##
##     Change DF before here     ##
##                               ##
##                               ##
##                               ##
###################################

X = df_baseline.PLAYER_ID

In [51]:
X = X.values.reshape(-1, 1)
# X_reshape=X.reshape(-1, 1)
# X = df_baseline.loc[:, basic]
X = MinMaxScaler().fit_transform(X)
# Y = df_baseline['FPTS'].values.reshape(-1,1).flatten()
Y = df_baseline.values.reshape(-1,1).flatten() # Y is 38 times larger. Not sure what I did here. 
Y = Y.reshape(-1, 1) 
Y = Y.reshape((16894, 38)) # Y is 38 times larger. Not sure what I did here. 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=101)

lin_reg = LinearRegression()
# rf=RandomForestClassifier(max_depth=8,n_estimators=5)

# scores = cross_validate(lasso, X, Y, cv=3, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)


reg_cv_score=cross_val_score(estimator=lin_reg,X=X_train,y=Y_train,cv=5)
print(reg_cv_score)

# errors = utils.cross_val(reg, X, y, n_folds=5, verbose=0)
# utils.summarize_errors(errors)

[nan nan nan nan nan]


The model has been built and trained on 2/3 of the data with a test on 1/3 of the data. 

In [49]:
# When the dataframes are combined, use this code to select features.

features = ['SG', 'F', 'C', 'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT',
            'FTA', 'FGA', '3PA', 'DRB', 'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc',
            'ORB_perc', 'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']

In [ ]:
_all = ['Salary', 'Rest', 'Rota_All', 'Rota_Pos', 'Home', 'SG', 'F', 'C', 'Value', 'FPTS_std',
        'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT', 'FTA', 'FGA', '3PA', 'DRB',
        'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc', 'TOV_perc', 
        'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']

In [55]:
### This should work when we start passing numbers. It will help pick best features.


# X was called above

X = MinMaxScaler().fit_transform(X)
# y = df_features['FPTS'].values.reshape(-1,1).flatten()

Y = df_baseline.values.reshape(-1,1).flatten() # Y is 38 times larger. Not sure what I did here. 
Y = Y.reshape(-1, 1) 
Y = Y.reshape((16894, 38)) # Y is 38 times larger. Not sure what I did here. 

# Takes 2 minutes
# clf.set_params(n_estimators=2000)
# clf.fit(X, y, sample_weight=train_weight)

model = GradientBoostingRegressor()
model.fit(X, Y)

top_features = pd.Series(model.feature_importances_, index = _all).sort_values()
top_features.plot(kind = "barh", figsize=(15,10) ,title='Top Features')
plt.show()

ValueError: ignored

In [ ]:
omit_lowest = 20
_selected = list(top_features[omit_lowest:].index)

# Building the Keras Model

In [61]:
tf.keras.backend.set_floatx('float64')

# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)


# x = df_baseline
# x = tf.ones((3, 3))
Y = model(X)

print(Y)

tf.Tensor(
[[ 2.91940524e-07  3.08297296e-07 -3.65564912e-07 -7.21618691e-07]
 [ 2.91940524e-07  3.08297296e-07 -3.65564912e-07 -7.21618691e-07]
 [ 2.70738259e-05  2.85907116e-05 -3.39015655e-05 -6.69210926e-05]
 ...
 [ 2.18450302e-04  2.30689582e-04 -2.73541215e-04 -5.39965535e-04]
 [ 2.70726179e-05  2.85894359e-05 -3.39000528e-05 -6.69181066e-05]
 [ 2.18703317e-04  2.30956773e-04 -2.73858038e-04 -5.40590938e-04]], shape=(16894, 4), dtype=float64)


Insert Layers

In [62]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
# x = df_baseline
Y = layer3(layer2(layer1(X)))
print(Y)

tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(16894, 4), dtype=float64)


# Part 3 Conclusions

This is where the conclusions section will be typed